In [3]:
# html 코드 그대로 크롤링 할시 주석도 같이 추출됨

from bs4 import BeautifulSoup
import sys
import requests
import re
import pandas as pd
import csv
import os
import openai

f_name = "신소재공학과_data"

openai.api_key ="API_KEY"

def split_sentences(text):
    sentences = text.split('\n')  # newline('\n')을 기준으로 문자열을 나눕니다.
    sentences = [sentence.strip() for sentence in sentences]  # 각 문장 앞뒤에 있는 공백을 제거합니다.
    sentences = [sentence for sentence in sentences if sentence]  # 빈 문장을 제거합니다.
    return sentences

def extract_tables(html_str):
    # 정규표현식 패턴 정의
    pattern = r'(<table.*?>.*?</table>)'
    # 정규표현식으로 <table>...</table> 부분 찾기
    tables = re.findall(pattern, html_str, re.DOTALL)  # re.DOTALL은 줄바꿈 문자도 포함해서 찾도록 합니다.
    return tables

def table_function(table_g):
    sys = f"""다음의 맥락들을 사용하여 마지막 답변을 생성하세요. 답변을 꾸며내려 하지 마십시오.
답변을 제공할 때는 다음 형식으로 제공되어야 합니다:
테이블: [테이블 태그와 내용이 주어짐]
답: [테이블 내용 반환]

답을 결정하는 방법:

- 테이블에 있는 내용으로만 답변을 생성합니다.
- 충분한 수준의 자세한 내용을 포함하여 답변을 생성합니다.
- 자신감이 과하지 않도록 주의하세요!

예시 #1
테이블:
---------
<table class="table_nrm center">
    <tbody><tr>
        <th>이수구분</th>
        <th>학점인정</th>
        <th>비고</th>
    </tr>
    <tr>
        <td>교양 선택</td>
        <td>1학점</td>
        <td>“P”(Pass), “NP”(Non-Pass)</td>
    </tr>
</tbody></table>
---------
답 : 
---------
이수구분 : 교양선택
학점인정 : 1학점
비고 :“P”(Pass), “NP”(Non-Pass)
---------

예시 #2
테이블:
---------
<table class="table_nrm center">
    <tbody><tr>
        <th>대영역</th>
        <th>세부영역</th>
        <th colspan="2">학점</th>
    </tr>
    <tr>
        <td>필수교양</td>
        <td>
            국어<br>
            영어<br>
            소프트웨어<br>
            항공우주특성화
        </td>
        <td colspan="2">
            공과대학 13 학점<br>
            항경대학 12 학점
        </td>
    </tr>
    <tr>
        <td>전공기초교양</td>
        <td>전공기초</td>
        <td colspan="2">0 ~ 18학점, 각 학부(과)별 상이</td>
    </tr>
    <tr>
        <td>핵심교양</td>
        <td>
            1) 철학과 역사<br>
            2) 문학과 예술<br>
            3) 인간과 사회<br>
            4) 자연과 과학<br>
            5) 실용 및 글로벌
        </td>
        <td>9 학점 이상<br>학부(과)별 지정영역이수<br>(4-1 참조)</td>
        <td rowspan="2">12 학점<br>※ 1) ~ 5) 학부(과)별 지정 영역 중<br>3개 선택 후 중복되지 않는 영역에서 3학점 이상 이수<br>※ 세부영역 중복, 대영역 미중복시 인정됨</td>
    </tr>
    <tr>
        <td>심화교양</td>
        <td>
            6) 철학과 역사<br>
            7) 문학과 예술<br>
            8) 인간과 사회<br>
            9) 자연과 과학<br>
            10) 실용 및 글로벌<br>
            11) 봉사와 리더십<br>
            12) 자기계발<br>
            13) 외국어(영어)
        </td>
        <td>자유이수</td>
    </tr>
    <tr>
        <th colspan="2">계</th>
        <th colspan="2">24 ~ 43 학점</th>
    </tr>
</tbody></table>
---------
답 : 
---------
대영역 : 필수교양
세부영역 : 국어, 영어, 소프트웨어, 항공우주특성화
학점 : 공과대학 13 학점, 항경대학 12 학점

대영역 : 전공기초교양
세부영역 : 전공기초
학점 : 0 ~ 18학점, 각 학부(과)별 상이

대영역 : 핵심교양
세부영역 : 
1) 철학과 역사
2) 문학과 예술
3) 인간과 사회
4) 자연과 과학
5) 실용 및 글로벌
학점 : 9 학점 이상, 학부(과)별 지정영역이수 (4-1 참조)
비고 : 12 학점(1) ~ 5) 학부(과)별 지정 영역 중 3개 선택 후 중복되지 않는 영역에서 3학점 이상 이수, 세부영역 중복, 대영역 미중복시 인정됨

대영역 : 심화교양
세부영역 : 
6) 철학과 역사
7) 문학과 예술
8) 인간과 사회
9) 자연과 과학
10) 실용 및 글로벌
11) 봉사와 리더십
12) 자기계발
13) 외국어(영어)
학점 : 자유이수
비고 : 12 학점(1) ~ 5) 학부(과)별 지정 영역 중 3개 선택 후 중복되지 않는 영역에서 3학점 이상 이수, 세부영역 중복, 대영역 미중복시 인정됨

계 : 24 ~ 43 학점
---------

예시 #3
테이블:
---------
<table class="table_nrm">
    <tbody><tr>
        <th colspan="2">구분</th>
        <th>비고</th>
    </tr>
    <tr>
        <td class="ta_center">공통서류</td>
        <td class="ta_center">전과신청서 각 1부<br>성적증명서 1부</td>
        <td class="ta_center" rowspan="3">전과 희망 학부(과) 사무실</td>
    </tr>
    <tr>
        <td class="ta_center">경영학부<br>항공교통물류학부</td>
        <td class="ta_center">토익성적증명서 1부</td>
    </tr>
    <tr>
        <td class="ta_center">항공운항학과</td>
        <td class="ta_center">전과지원서 1부<br>토익성적증명서 1부<br>항공신체검사증명서 1부</td>
    </tr>
</tbody></table>
---------
답 : 
---------
구분 : 공통서류 - 전과신청서 각 1부, 성적증명서 1부
비고 : 전과 희망 학부(과) 사무실

구분 : 경영학부, 항공교통물류학부 - 토익성적증명서 1부
비고 : 전과 희망 학부(과) 사무실

구분 : 항공운항학과 - 전과지원서 1부, 토익성적증명서 1부, 항공신체검사증명서 1부
비고 : 전과 희망 학부(과) 사무실
---------

시작!
테이블:
---------
{table_g}
---------
답 : 
"""
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo-16k-0613",
      messages=[
      {"role": "system", "content": f'"{sys}"'},
    ], temperature =  0.001)
    answer = completion['choices'][0]['message']['content']
    if answer.endswith('"'):
        answer = answer[:-1]
        answer = answer
    return answer 

# 홈페이지 주소 가져오기
# url = "https://kau.ac.kr/web/pages/gc15707h.do"
url = """http://college.kau.ac.kr/web/pages/gc14812h.do
http://college.kau.ac.kr/web/pages/gc60194h.do
http://college.kau.ac.kr/web/pages/gc73779h.do
http://college.kau.ac.kr/web/pages/gc2790h.do
http://college.kau.ac.kr/web/pages/gc6693h.do
http://college.kau.ac.kr/web/pages/gc31731h.do
http://college.kau.ac.kr/web/pages/gc51159h.do
http://college.kau.ac.kr/web/pages/gc63809h.do
http://college.kau.ac.kr/web/pages/gc31237h.do
http://college.kau.ac.kr/web/pages/gc94566h.do
http://college.kau.ac.kr/web/pages/gc21472h.do
http://college.kau.ac.kr/web/pages/gc20791h.do
http://college.kau.ac.kr/web/pages/gc2940h.do
http://college.kau.ac.kr/web/pages/gc76425h.do
http://college.kau.ac.kr/web/pages/gc11672h.do
http://college.kau.ac.kr/web/pages/gc13005h.do
http://college.kau.ac.kr/web/pages/gc57782h.do
http://college.kau.ac.kr/web/pages/gc33696h.do
http://college.kau.ac.kr/web/pages/gc92360h.do
http://college.kau.ac.kr/web/pages/gc24861h.do
http://college.kau.ac.kr/web/pages/gc86306h.do
"""
url_split = split_sentences(url)
url_list = []

for i in range(len(url_split)):
    url_list.append(url_split[i].strip())
print(url_list)

for jj in range(len(url_list)):
    if jj == 11:
        f = open(f'{f_name}.csv','a', newline='' , encoding="utf-8-sig")
        answer_db = csv.writer(f)
        answer_db.writerow([str(jj), "", url_list[jj]])
        f.close()
        continue
    temp_url = url_list[jj]
    html = requests.get(temp_url)
    bs_html = BeautifulSoup(html.content, "html.parser")

    # print(bs_html) # 웹 페이지 출력
    bs_html = str(bs_html)
    split_text = split_sentences(bs_html)
    tp = 0
    tp_noise = 0
    tp_annotation = 0
    temp_list = []
    for i in range(len(split_text)):
        temp_text = split_text[i]
        if "<html lang" in temp_text and tp < 5:
            tp += 1
        elif "<html lang" in temp_text and tp == 5:
            temp_list.append(temp_text)
#             print(temp_text)
        elif tp == 5 and "</html>" in temp_text:
            break
        elif tp == 5 and "<!--" in temp_text and "-->" in temp_text:
            temp_text = re.sub(r'<!--.*?-->', '', temp_text)
            temp_list.append(temp_text)            
        elif tp == 5 and "<!--" in temp_text and "-->" not in temp_text:
            tp_annotation = 1
            continue
        elif tp == 5 and tp_annotation == 1 and "-->" not in temp_text:
            continue
        elif tp == 5 and tp_annotation == 1 and "-->" in temp_text:
            tp_annotation = 0
            continue
        elif tp == 5:
            temp_list.append(temp_text)
#             print(temp_text)

    print("%%%%%%%%%%%%%%%%%%%")
    
    # table 태그는 하나의 리스트 원소로 두기
    
    table_tp = 0
    table_temp_str = ""
    re_table_list = []
    for i in range(len(temp_list)):
        if "<table" in temp_list[i] and  "</table>" in temp_list[i] and table_tp == 0:  # 한 줄 table 일 때
            test_str = temp_list[i]
            tables = extract_tables(test_str)
            for table in tables:
                # 해당 table이 test_str에서 시작하는 위치 찾기
                start_index = test_str.find(table)
                # 해당 table의 내용을 출력하기 전까지의 test_str의 내용을 출력
                re_table_list.append(test_str[:start_index].strip())
                # 해당 table 출력
                re_table_list.append(table.strip())
                # 이미 출력된 부분을 test_str에서 제거
                test_str = test_str[start_index + len(table):]
                # 남은 test_str에서 내용 출력
            re_table_list.append(test_str.strip())
        elif "<table" in temp_list[i] and table_tp == 0:
            table_tp = 1
            table_temp_str = table_temp_str + temp_list[i] + "\n"            
        elif "</table>" not in temp_list[i] and table_tp == 1:
            table_temp_str = table_temp_str + temp_list[i] + "\n"
        elif "</table>" in temp_list[i] and table_tp == 1:
            table_tp = 0
            table_temp_str = table_temp_str + temp_list[i] + "\n"
            re_table_list.append(table_temp_str.strip())
            table_temp_str = ""
        else:
            re_table_list.append(temp_list[i])
    
    print(f"-------{jj}-------")
    print(len(re_table_list))
    print(re_table_list)
# #     # table 태그 임의로 변형 (미완성)
    
#     re_table_list2 = []
#     table_temp_str = ""
#     tr_tp = 0
#     for i in range(len(re_table_list)):
#         if "<table" in re_table_list[i]:
#             table_tag_context = split_sentences(re_table_list[i])
#             for j in range(len(table_tag_context):
#                 if "<tr>" not in table_tag_context[i]
#                     temp_line = re.sub('(<([^>]+)>)', '', table_tag_context[i])
#                     temp_line = temp_line.strip()    
#                     table_temp_str = table_temp_str + temp_line + "\n"
#                 elif "<tr>" in table_tag_context[i]
#                     tr_tp = 1
#                     temp_line = re.sub('(<([^>]+)>)', '', table_tag_context[i])
#                     temp_line = temp_line.strip()    
#                     table_temp_str = table_temp_str + temp_line
#         else:
#             re_table_list2.append(re_table_list[i])

#  table 태그는 gpt에게 정리
    re_table_list2 = []               
    for i in range(len(re_table_list)):
        if "<table" in re_table_list[i]:
            table_g = re_table_list[i]
            err_count = 0
            try :
                answer = table_function(table_g)
            except Exception as e:
                err_count += 1
                print(err_count)
                if err_count >= 5:
                    print("예외가 연속으로 5번 발생했습니다. 프로그램을 종료합니다.")
                else:
                    print("예외가 발생했습니다. (%s)" % e)
                    answer = table_function(table_g)        
            re_table_list2.append(answer.strip())
        else:
            re_table_list2.append(re_table_list[i])
            
    previous_line = "#$%"
    temp_str = ""
    for i in range(len(re_table_list2)):
        temp_line = re.sub('(<([^>]+)>)', '', re_table_list2[i])
        temp_line = temp_line.strip()
        if previous_line == "" and temp_line == "":
            continue
        else:
            print(temp_line)
            temp_str = temp_str + temp_line + "\n"
        previous_line = temp_line
    temp_str = temp_str.strip()
    f = open(f'{f_name}.csv','a', newline='' , encoding="utf-8-sig")
    answer_db = csv.writer(f)
    answer_db.writerow([str(jj), temp_str, url_list[jj]])
    f.close()

['http://college.kau.ac.kr/web/pages/gc14812h.do', 'http://college.kau.ac.kr/web/pages/gc60194h.do', 'http://college.kau.ac.kr/web/pages/gc73779h.do', 'http://college.kau.ac.kr/web/pages/gc2790h.do', 'http://college.kau.ac.kr/web/pages/gc6693h.do', 'http://college.kau.ac.kr/web/pages/gc31731h.do', 'http://college.kau.ac.kr/web/pages/gc51159h.do', 'http://college.kau.ac.kr/web/pages/gc63809h.do', 'http://college.kau.ac.kr/web/pages/gc31237h.do', 'http://college.kau.ac.kr/web/pages/gc94566h.do', 'http://college.kau.ac.kr/web/pages/gc21472h.do', 'http://college.kau.ac.kr/web/pages/gc20791h.do', 'http://college.kau.ac.kr/web/pages/gc2940h.do', 'http://college.kau.ac.kr/web/pages/gc76425h.do', 'http://college.kau.ac.kr/web/pages/gc11672h.do', 'http://college.kau.ac.kr/web/pages/gc13005h.do', 'http://college.kau.ac.kr/web/pages/gc57782h.do', 'http://college.kau.ac.kr/web/pages/gc33696h.do', 'http://college.kau.ac.kr/web/pages/gc92360h.do', 'http://college.kau.ac.kr/web/pages/gc24861h.do', 'h

%%%%%%%%%%%%%%%%%%%
-------3-------
14
['<head>', '</head>', '<script type="text/javascript">', '$(document).ready(function() {', '});', '</script>', '<body>', '<div class="sub_conts">', '<div class="sub_tit">', '<h3>교육목표</h3>', '</div>', '한국항공대학교 신소재공학과는 대학의 인재상인 “창의적 인재”, “실용적 인재”, “지도적 인재”를 신소재공학의 학문 범위 내에서 양성하기 위해 다음의 교육목표를 설정하고 있습니다.\xa0<table class="table_01"><colgroup><col style="width:15%;"/><col style="width: 50%;"/><col style="width: 5%;"/></colgroup><tbody><tr><th style="text-align: center;">창의적 인재</th><td style="text-align: left;"><div align="justify" style="text-align: justify;"><div align="center" style="text-align: center;">입체적 과학기반교육을 바탕으로 미래 첨단소재 강국을 이끌어 갈</div><b><div align="center" style="text-align: center;"><b>창의적 과학인의 양성</b></div></b></div></td></tr><tr><th>실용적 인재</th><td style="text-align: left;"><div align="center" style="text-align: center;">신소재 공학 중심의 전문교육을 바탕으로 미래산업을 주도하는</div><b><div align="center" style="text-align: center;"><b>실용적 공학인의 양성</b></div></b></td


$(document).ready(function() {
});

신소재공학과 운영내규제 1 장 총 칙제1조 (목적)이 운영내규는 신소재공학과(이하 '본 학부'로 칭함)의 교육 목표를 달성하기 위한 합리적인 학부 운영을 위해 필요한 사항을 규정함을 목적으로 한다. &lt;신설 2008.6.1, 개정 2021.03.01&gt;  제 2 장 학부위원회제2조 (위원회 구성
및 역할)
① 본
신소재공학과(이하, 본 학과라 칭함)에는 운영위원회, 교과과정위원회, 평가위원회, 외부자문위원회 및 학생지도위원회를 둔다.&lt;개정 2021.03.01&gt;
② &lt;삭제 2019. 3. 25&gt;
③ 운영위원회는
위원장과 위원으로 구성되며 위원장은 학과장으로 한다. 위원은 본 학과에 설치된 모든 위원회의
위원장을 당연직 위원으로 하고, 약간 명의 순환직 위원을 본 학과 교수 중에서 임명하여 구성한다.
④ 다른
위원회의 위원장 및 위원은 운영위원회에서 정하는 방법에 따른다.
제 3 장 재입학 및 학기재수제3조 (재입학 심사)재입학 심사는 학과
재입학 심의위원회에서 심층면접을 통하여 심사하며, 그 내용은 아래의 항과 같다.
① 재입학
심의위원회는 학과장 1인을 포함한 3인의 위원으로
구성한다.
② 심층면접은
재입학 신청자의 수학능력여부, 잔여재학기간 내 졸업가능여부 등을 종합 심사하며, 면접방식 및 출제범위는 아래의 각 호와 같다.
1. 심층면접은 본 학과 전공기초학문분야에 대한 지식을 평가하기 위한 구두평가 방식으로 실시한다.
2. 구체적인 출제범위와 방법은 학과교수회의에서 결정하며, 별도의 공고가 없으면
직전학기에 준하여 시행한다.
&lt;전문개정 2008.10.27&gt;
제4조 (학기재수
및 과목재수)학기재수 및 과목재수는
학칙 제25조, 학칙 시행세칙 제29조에 따른다.  제 4 장 교과과정제5조 (교과과정
편성 및 심의)교과과정 편성은 학칙
제 18조에 따른다.
① 교과과정
편성의 필요성이 있다고 판단될 경우에는 교과과정개편(안)을
교과과정위원회에서 심의한다. &

%%%%%%%%%%%%%%%%%%%
-------9-------
181
['<head>', '</head>', '<script type="text/javascript">', '$(document).ready(function() {', '});', '</script>', '<body>', '<div class="sub_conts">', '<link href="https://fonts.googleapis.com/css2?family=Gothic+A1:wght@400;500;600;700&amp;family=Noto+Sans+KR:wght@500&amp;family=Noto+Serif+KR&amp;family=Poppins:ital,wght@0,500;0,700;1,700&amp;display=swap" rel="stylesheet"/>', '<div class="sub_tit">', '<h3><p class="" style="line-height: 2; word-break: keep-all; margin: 0cm 0cm 8pt; text-align: justify; font-size: 10pt; font-family: " 고딕";"="" 맑은=""><span courier="" new";"="" style=\'font-size: 12pt; color: rgb(0, 0, 0); font-family: "Courier New";\'>교과목 설명</span><span lang="EN-US" style="font-weight: normal;"><o:p></o:p></span></p><p class="" style="line-height: 2; word-break: keep-all; margin: 0cm 0cm 8pt; text-align: justify; font-size: 10pt; font-family: " 고딕";"="" 맑은=""><span courier="" lang="EN-US" new";"="" style=\'font-weight: normal; color:


$(document).ready(function() {
});

교과목 설명

교과목명 : 신소재공학설계입문
과목설명 : 공학설계교육의 첫 단계로써 공학설계프로세스를 이해하고, 팀활동을 통해 직접 수행하는 과정을 통해 아이디어 도출과 공학적 검증 능력을 배양한다.

교과목명 : 미래산업과 재료기술
과목설명 : 다양한 미래 첨단 산업의 핵심 기술인 재료(소재)기술에 대한 입문 과정으로 교양수준의 기본 개념 강의와 국내외 전문가들의 특강을 병행하여 아직 전공과정에 진입하지 않은 1학년 학생들이 재료공학의 분야에 대한 기대와 올바른 관점을 정립하는 데 필요한 지식을 전달한다.

교과목명 : 재료과학Ⅰ
과목설명 : 재료의 기초이론과 고상의 구조, 특성, 고체 내부에서의 원자의 규칙 및 불규칙성을 이해시키고 단상금속의 탄, 소성변형 그리고 열처리에 따른 구조 및 성질변화를 다룬다.

교과목명 : 재료과학Ⅱ
과목설명 : 재료공학을 입문하는 학생에게 재료공학 전분야에 대하여 기초적인 지식을 습득하게 한다. 재료과학Ⅱ에서는 세라믹과 재료의 전자적, 공학적 특성과 자성재료에 대하여 강의한다.

교과목명 : 재료열역학Ⅰ
과목설명 : 열역학에 대한 기본적인 용어의 정의와 이들의 응용과 열과 일의 전환, 열역학 제1, 2법칙에 관한 이론과 응용을 강의하고 재료의 상변태등을 이해하는데 필수적인 기초 이론과 열역학관계식들에 관하여 강의한다.

교과목명 : 재료열역학Ⅱ
과목설명 : 열역학의 기본법칙들과 가스 고체사이의 반응 용액의 거동 등을 이해시키고 상변태와 재질변화와의 상관관계, 상변태에 대한 열역학적 해석을 설명하고 실제 재료에서의 상변태에 대한 변태 기구를 분한다.

교과목명 : 공업수학
과목설명 : 재료공학에서 필요한 상미분방정식과 벡터를 다룸으로서 수학적 개념을 공업적으로 응용할 수 있도록 한다.

교과목명 : 결정구조론
과목설명 : 결정기하학에 관한 결정계, Bravis 격자, 대칭성을 연구하고 X-ray에 관한 실험 방법들과 특징들을 강의한다.

교과목명 : 재료물리
과


$(document).ready(function() {
});

복합재료제조공정연구실
Composite Manufacturing Technology Lab.

지도교수 : 최원종 (Choi, Won Jong)
연구실 위치 : 비행교육원 격납고 (지도교수), 기계관 221호 (대학원생)
연락처 : Email : wjchoi@kau.ac.kr
실험실 : 비행교육원 격납고(1층)
홈페이지 주소 : 없음

연구분야 소개 : 복합재료제조공정연구실은 CFRP, GFRP 등의 복합재료 소재를 기반으로, 항공우주산업에서 널리 활용 중에 있는 Autoclave Process는 물론 RTM, PCM 등 최신의 OoA Process 공정기술의 개발 및 최적화를 통한 주력산업의 고도화와 미래선도산업에서의 핵심소재로 분류되는 복합재료의 응용기술 개발을 목표로 연구를 진행하고 있습니다.

주요연구 내용 : 이미지 참조

%%%%%%%%%%%%%%%%%%%
-------13-------
21
['<head>', '</head>', '<script type="text/javascript">', '$(document).ready(function() {', '});', '</script>', '<body>', '<div class="sub_conts">', '<div class="sub_tit">', '<h3>표면 및 박막공학연구실</h3>', '<p>Surface &amp; Thin Film Lab.</p>', '</div>', '<div class="terms">', '<div class="chpt">', '<div class="artc">', '<table class="table_01">\n<colgroup>\n<col style="width: 20%;"/>\n</colgroup>\n<tbody>\n<tr>\n<th>지도교수</th>\n<td style="text-align: left;">송요승</td>\n</tr>\n<tr>\n<th>연구실 위치</th>\n<td style="tex


$(document).ready(function() {
});

표면기술 응용 연구 센터
Center for Surface Technology Application (CeSTA)

지도교수 : 이상율 (Lee, Sang-Yul)
연구실 위치 : 기계관 417호 (지도교수), 기계관 224호 (대학원생)
연락처 : Email : sylee@kau.ac.kr, Tel : 02-300-0166
실험실 : 연구동 309호, 기계관 202-B호

연구분야 소개 : 표면기술 응용 연구 센터에서는 금속 및 비금속 물질에 대한 표면처리 공정 기술 개발 및 응용에 관한 연구를 중점적으로 진행하고 있습니다. 주 연구 분야로는 진공 플라즈마 공정을 활용한 인공위성용 태양전지 보호용 탄소 박막 개발 및 자동차용 고내식 스마트 강판 개발과 공구/금형 분야 적용 고경도 경질 박막 연구 이며, 기초 공정/구조 엔지니어링을 통한 합성 및 분석을 통해 다양한 목적에 부합한 차세대 소재 기술 개발 및 응용에 대한 연구를 수행하고 있습니다.

주요연구 내용 : [이미지 참조]

%%%%%%%%%%%%%%%%%%%
-------15-------
21
['<head>', '</head>', '<script type="text/javascript">', '$(document).ready(function() {', '});', '</script>', '<body>', '<div class="sub_conts">', '<div class="sub_tit">', '<h3>친환경 경금속재료 연구실</h3>', '<p>Eco-Light Metallic Materials Lab(ELMML)</p>', '</div>', '<div class="terms">', '<div class="chpt">', '<div class="artc">', '<table class="table_01">\n<colgroup>\n<col style="width: 20%;"/>\n</colgroup>\n<tbody>\n<tr>\n<t


$(document).ready(function() {
});

비정질 재료 분석 연구실
Non-crystalline Materials Analysis Lab (NoMAL)

지도교수 : 최용규 (Choi, Yong Gyu)
연구실 위치 : 기계관 416호 (지도교수), 기계관 216-1호 (대학원생)
연락처 : Email : ygchoi@kau.ac.kr, Tel : 02-300-0169
실험실 : 기계관 116, 203A, 302-1, 연구동 408

연구분야 소개 : 비정질 재료 분석 연구실에서는 주로 유리소재 및 비정질 박막소재의 조성-구조-물성의 상관관계를 연구합니다. 주로 광학적 또는 전기적 물성의 제어를 통하여 원적외선 결상용 렌즈, 중적외선 레이저, 백색광 LED용 형광체, 상변화 메모리 등의 응용을 위한 기능성 소재를 개발합니다. 또한, 새로운 방식의 이온교환 공정을 활용하여 초박형 유리를 위시한 광학 유리 연구도 진행하고 있으며, 방사광가속기 기반 X선 흡수분광 분석기법을 통한 비정질 재료의 원자배열 구조해석 연구도 수행하고 있습니다.

주요연구 내용 : 
- 유리소재 및 비정질 박막소재의 조성-구조-물성의 상관관계 연구
- 광학적 또는 전기적 물성의 제어를 통한 기능성 소재 개발
- 새로운 방식의 이온교환 공정을 활용한 광학 유리 연구
- 방사광가속기 기반 X선 흡수분광 분석기법을 통한 비정질 재료의 원자배열 구조해석 연구

%%%%%%%%%%%%%%%%%%%
-------17-------
21
['<head>', '</head>', '<script type="text/javascript">', '$(document).ready(function() {', '});', '</script>', '<body>', '<div class="sub_conts">', '<div class="sub_tit">', '<h3>디스플레이 소재 연구실 </h3>', '<p>Display Materials Lab.</p>', '</div>', '<div clas


$(document).ready(function() {
});

전자 및 에너지소자재료 연구실
Electronic &amp; Energy Device Materials Lab. (E^2DML)

지도교수 : 권도균 Kwon, Do-Kyun
연구실 위치 : 기계관 415호 (지도교수), 기계관 221호 (대학원생)
연락처 : Email : dkwon@kau.ac.kr, Tel : 02-300-0164
실험실 : 연구동 207호
홈페이지 : https://sites.google.com/view/kau-e2dml

연구분야 소개 : 전자 및 에너지소자재료 연구실에서는 고체 물질의 전기적 성질에 기반한 소자 응용과 물성 증진을 위한 재료 합성 및 공정을 종합적으로 연구하고 있습니다.

주요연구 내용 : [이미지 참조]

%%%%%%%%%%%%%%%%%%%
-------19-------
21
['<head>', '</head>', '<script type="text/javascript">', '$(document).ready(function() {', '});', '</script>', '<body>', '<div class="sub_conts">', '<div class="sub_tit">', '<strong> </strong><h3> 반도체 소재 및 소자 연구실</h3><strong>', '</strong><p>Semiconductor materials and Devices Lab.\u200b</p>', '</div>', '<div class="terms">', '<div class="chpt">', '<div class="artc">', '<table class="table_01">\n<colgroup>\n<col style="width: 20%;"/>\n</colgroup>\n<tbody>\n<tr>\n<th>지도교수</th>\n<td style="text-align: left;">황완식\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\


$(document).ready(function() {
});

레이저 및 광전자소자 재료 연구실
Laser and Optoelectronic Device Materials Lab (LODML)

지도교수 : 윤요한 Yoon, Yohan
연구실 위치 : 기계관 414호 (지도교수), 기계관 224호 (대학원생)
연락처 : Email : yyoon@kau.ac.kr, Tel : 02-300-0162
실험실 : 연구동 311호
홈페이지 주소 : 

연구분야 소개 : 
레이저 및 광전자소자 재료 연구실에서는 spectroscopy를 기반으로 한 여러가지 전기적, 광학적 테크닉을 이용하여 재료, 소자 분석의 프로토콜의 개발을 주요 목표로 하고 있습니다. 그 연구분야는 크게 두가지로 나뉠 수 있습니다.

1. Scanning probe microscopy (SPM)-based techniques (Near-field scanning microscope (NSOM), Photothermal induced resonance (PTIR))을 이용한 photovoltaic devices의 전기적, 광학적 성분 분석.

2. Near IR, IR laser를 이용한 폭발물, 가스등의 원거리 탐지 기술 개발 (Standoff detection of trace explosives and gas using near IR and IR lasers).

3. broadband lamp를 이용한 고화질 바이오 이미징 기술 개발 (high resolution bioimaging using UV-Vis light source).

4. Broadband lamp와 CMOS 카메라를 이용한 실시간 분광기술 개발 (Real-time spectroscopy using UV-Vis light source & CMOS camera).

주요연구 내용 : 
[이미지]



In [ ]:
import re
import csv
import pandas as pd


def split_sentences(text):
    sentences = text.split('\n')  # newline('\n')을 기준으로 문자열을 나눕니다.
#     sentences = [sentence.strip() for sentence in sentences]  # 각 문장 앞뒤에 있는 공백을 제거합니다.
#     sentences = [sentence for sentence in sentences if sentence]  # 빈 문장을 제거합니다.
    return sentences


page_indexList = []
pageList = []
data = pd.read_csv('test3.csv', header=None, encoding="UTF-8")
for i in range(len(data)):
    page_indexList.append(data.iloc[i][0])
    pageList.append(data.iloc[i][1])

temp_str = ""
for i in range(len(page_indexList)):
    if page_indexList[i] == 41:
        temp_page = split_sentences(pageList[i])
        for j in range(len(temp_page)):
            if "// 팝업 띄우기" in temp_page[j]:
                temp_str = temp_str.strip()
                break
            else:
                temp_str = temp_str + temp_page[j] + "\n"
        temp_str = temp_str.strip()
        f = open(f'test3_filter.csv','a', newline='' , encoding="UTF-8")
        answer_db = csv.writer(f)
        answer_db.writerow([page_indexList[i], temp_str])
        f.close()    
        temp_str = ""
    else:
        f = open(f'test3_filter.csv','a', newline='' , encoding="UTF-8")
        answer_db = csv.writer(f)
        answer_db.writerow([page_indexList[i], pageList[i]])
        f.close()    
        
    

In [ ]:
import os
import openai

openai.api_key ="API_KEY"
table_g = """                            <table class="table_01"><tbody><tr><th>대영역</th><th>세부영역</th><th colspan="2">학점</th></tr><tr><td>공통교양</td><td>-</td><td colspan="2">10~13학점</td></tr><tr><td>학문기초교양</td><td>-</td><td colspan="2">13~21학점</td></tr><tr><td rowspan="2">선택교양</td><td>1) 철학과 역사<br>2) 문학과 예술<br>3) 인간과 사회<br>4) 자연과 과학<br>5)세계와 문화</td><td>9 학점 이상<br>(최소 3개 영역 선택)</td><td rowspan="2">12 학점<br>※ 1)~5) 영역 중 3개 선택 후 중복되지 않는 나머지 4개 영역에서 3학점 이상 이수</td></tr><tr><td>6) 소양<br>7) 자기계발</td><td>자유이수</td></tr><tr><td colspan="2">계</td><td colspan="2">39 학점 이상</td></tr></tbody></table></div></h5><h5>1-1) 2023학번 학부(과) 별 교양이수학점</h5><h5><div class="scroller"><table class="table_01"><tbody><tr><th>학부/과</th><th>공통교양</th><th>학문기초교양</th><th>선택교양</th><th>추가이수<br>교양학점*</th><th>계</th></tr><tr><td>항공우주 및 기계공학부</td><td>11</td><td>21</td><td>12</td><td>-</td><td>44</td></tr><tr><td>항공전자정보공학부</td><td>11</td><td>20</td><td>12</td><td>-</td><td>43</td></tr><tr><td>신소재공학과</td><td>11</td><td>17</td><td>12</td><td>-</td><td>40</td></tr><tr><td>소프트웨어학과</td><td>13</td><td>13</td><td>12</td><td>1</td><td>39</td></tr><tr><td>스마트드론공학과</td><td>10</td><td>19</td><td>12</td><td>-</td><td>41</td></tr><tr><td>AI자율주행시스템공학과</td><td>12</td><td>16</td><td>12</td><td>-</td><td>40</td></tr><tr><td>항공교통물류학부</td><td>11</td><td>15</td><td>12</td><td>1</td><td>39</td></tr><tr><td>항공운항학과</td><td>11</td><td>15</td><td>12</td><td>1</td><td>39</td></tr><tr><td>경영학부</td><td>11</td><td>15</td><td>12</td><td>1</td><td>39</td></tr></tbody></table></div></h5>* 졸업에 필요한 추가이수 교양학점은 공통교양, 학문기초교양, 선택교양 모든 교과목 포함됨<strong>&nbsp;&nbsp;&nbsp;<br><span style="color: rgb(0, 0, 0);"><span style="color: rgb(58, 50, 195);">※ 년도별 교양교과과정표를 확인하여 소속 학부(과)별 필수이수 과목을 확인할 것</span>(</span></strong><a href="http://college.kau.ac.kr/web/index.do?siteFlag=ct_www"><span style="color: rgb(0, 0, 0);">한국항공대학교 인문자연학부 (kau.ac.kr)</span></a><span style="color: rgb(0, 0, 0);">)</span><strong><br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </strong><h5><br>2) 2022학번<br><br> <div class="scroller"><table class="table_01"><tbody><tr><th>대영역</th><th>세부영역</th><th colspan="2">학점</th></tr><tr><td>공통교양</td><td>- </td><td colspan="2">11~13학점</td></tr><tr><td>학문기초교양</td><td>- </td><td colspan="2">12~20학점</td></tr><tr><td rowspan="2">선택교양</td><td>1) 철학과 역사<br>2) 문학과 예술<br>3) 인간과 사회<br>4) 자연과 과학<br>5)세계와 문화</td><td>9 학점 이상<br>(최소 3개 영역 선택)</td><td rowspan="2">12 학점<br>※ 1)~5) 영역 중 3개 선택 후 중복되지 않는 나머지 4개 영역에서 3학점 이상 이수</td></tr><tr><td>6) 소양     <br>7) 자기계발</td><td>자유이수</td></tr><tr><td colspan="2">계</td><td colspan="2">39 학점 이상</td></tr></tbody></table></div><!--2-1) 학부 ( 과 ) 별 교양이수학점 -->     </h5><h5>2-1) 2022학번 학부(과) 별 교양이수학점</h5><h5><div class="scroller"><table class="table_01"><tbody><tr><th>학부/과</th><th>공통교양</th><th>학문기초교양</th><th>선택교양</th><th>추가이수<br>교양학점*</th><th>계</th></tr><tr><td>항공우주 및 기계공학부</td><td>11</td><td> 20 </td><td> 12 </td><td>- </td><td> 43</td></tr><tr><td>항공전자정보공학부</td><td>11</td><td> 18</td><td> 12 </td><td> - </td><td>41</td></tr><tr><td>신소재공학과</td><td>11</td><td> 15</td><td> 12 </td><td>1</td><td> 39 </td></tr><tr><td>소프트웨어학과</td><td>13</td><td> 12</td><td> 12 </td><td>2</td><td> 39 </td></tr><tr><td>스마트드론공학과</td><td>12</td><td> 18</td><td> 12 </td><td>-</td><td>42</td></tr><tr><td>AI자율주행시스템공학과</td><td>12</td><td>15</td><td> 12 </td><td>-</td><td> 39 </td></tr><tr><td>항공교통물류학부</td><td>11</td><td> 13 </td><td> 12 </td><td>3</td><td> 39 </td></tr><tr><td>항공운항학과</td><td>11</td><td> 13 </td><td> 12 </td><td>3</td><td> 39 </td></tr><tr><td>경영학부</td><td>11</td><td> 13 </td><td> 12 </td><td>3</td><td> 39 </td></tr></tbody></table><p>* 졸업에 필요한 추가이수 교양학점은 공통교양, 학문기초교양, 선택교양 모든 교과목 포함됨<br><br><br>3) 2021학번<br><br> </p><div class="scroller"><table class="table_01"><tbody><tr><th>대영역</th><th>세부영역</th><th colspan="2">학점</th></tr><tr><td>공통교양</td><td>- </td><td colspan="2">9~11학점</td></tr><tr><td>학문기초교양</td><td>- </td><td colspan="2">12~20학점</td></tr><tr><td rowspan="2">선택교양</td><td>1) 철학과 역사<br>2) 문학과 예술<br>3) 인간과 사회<br>4) 자연과 과학<br>5)세계와 문화</td><td>9 학점 이상<br>(최소 3개 영역 선택)</td><td rowspan="2">12 학점<br>※ 1)~5) 영역 중 3개 선택 후 중복되지 않는 나머지 4개 영역에서 3학점 이상 이수</td></tr><tr><td>6) 소양     <br>7) 자기계발</td><td>자유이수</td></tr><tr><td colspan="2">계</td><td colspan="2">39 학점 이상</td></tr></tbody></table></div><!--2-1) 학부 ( 과 ) 별 교양이수학점 -->     </div></h5><h5>3-1) 2021학번 학부(과) 별 교양이수학점</h5><h5><div class="scroller"><table class="table_01"><tbody><tr><th>학부/과</th><th>공통교양</th><th>학문기초교양</th><th>선택교양</th><th>추가이수<br>교양학점*</th><th>계</th></tr><tr><td>항공우주 및 기계공학부</td><td>9 </td><td> 20 </td><td> 12 </td><td>- </td><td> 41</td></tr><tr><td>항공전자정보공학부</td><td>9 </td><td> 18</td><td> 12 </td><td>-</td><td> 39 </td></tr><tr><td>신소재공학과</td><td>9</td><td> 15</td><td> 12 </td><td>3</td><td> 39 </td></tr><tr><td>소프트웨어학과</td><td>11</td><td>12</td><td> 12 </td><td>4</td><td> 39 </td></tr><tr><td>스마트드론공학과</td><td>10</td><td>15</td><td> 12 </td><td>2</td><td> 39</td></tr><tr><td>항공교통물류학부</td><td>9</td><td> 13 </td><td> 12 </td><td> 5 </td><td> 39 </td></tr><tr><td>항공운항학과</td><td>9 </td><td> 13 </td><td> 12 </td><td> 5 </td><td> 39 </td></tr><tr><td>경영학부</td><td>9 </td><td> 13 </td><td> 12 </td><td> 5 </td><td> 39 </td></tr><tr></tr></tbody></table><p>* 졸업에 필요한 추가이수 교양학점은 공통교양, 학문기초교양, 선택교양 모든 교과목 포함됨<br><br>4) 2020 학번</p></div></h5><strong>
                            </strong><strong>
                                </strong><strong>
                                    </strong><strong>
                                    </strong><strong>
                                    </strong><table class="table_nrm center"><tbody><tr><th>대영역</th><th>세부영역</th><th colspan="2">학점</th>
                                </tr>
                                <tr>
                                    <td>공통교양</td>
                                    <td>-</td>
                                    <td colspan="2">7 학점</td>
                                </tr>
                                <tr>
                                    <td>학문기초교양</td>
                                    <td>-</td>
                                    <td colspan="2">12 ~ 20학점, 각 학부(과)별 상이</td>
                                </tr>
                                <tr>
                                    <td rowspan="2">선택교양</td>
                                    <td>
                                        1) 철학과 역사<br>
                                        2) 문학과 예술<br>
                                        3) 인간과 사회<br>
                                        4) 자연과 과학<br>
                                        5) 세계와 문화
                                    </td>
                                    <td>9 학점 이상<br>(최소 3개 영역 선택)</td>
                                    <td rowspan="2">12 학점<br>※ 1) ~ 5) 영역 중 3개 선택 후 중복되지 않는 영역에서 3학점 이상 이수</td>
                                </tr>
                                <tr>
                                    <td>
                                        6) 소양<br>
                                        5) 자기계발
                                    </td>
                                    <td>자유이수</td>
                                </tr>
                                <tr>
                                    <th colspan="2">계</th>
                                    <th colspan="2">39 학점 이상</th>
                                </tr>
                            </tbody></table>"""

sys = f"""다음의 맥락들을 사용하여 마지막 답변을 생성하세요. 답변을 꾸며내려 하지 마십시오.
답변을 제공할 때는 다음 형식으로 제공되어야 합니다:
테이블: [테이블 태그와 내용이 주어짐]
답: [테이블 내용 반환]

답을 결정하는 방법:

- 테이블에 있는 내용으로만 답변을 생성합니다.
- 충분한 수준의 자세한 내용을 포함하여 답변을 생성합니다.
- 자신감이 과하지 않도록 주의하세요!

예시 #1
테이블:
---------
<table class="table_nrm center">
    <tbody><tr>
        <th>이수구분</th>
        <th>학점인정</th>
        <th>비고</th>
    </tr>
    <tr>
        <td>교양 선택</td>
        <td>1학점</td>
        <td>“P”(Pass), “NP”(Non-Pass)</td>
    </tr>
</tbody></table>
---------
답 : 
---------
이수구분 : 교양선택
학점인정 : 1학점
비고 :“P”(Pass), “NP”(Non-Pass)
---------

예시 #2
테이블:
---------
<table class="table_nrm center">
<tbody>
  <tr>
    <th>편의시설</th>
    <th>위치</th>
    <th>전화번호</th>
  </tr>
  <tr>
    <td>식당</td>
    <td>1층 학생식당 내</td>
    <td>-</td>
  </tr>
  <tr>
    <td>편의점(GS)</td>
    <td>1층 학생식당 내</td>
    <td>02-3158-0336</td>
  </tr>
  <tr>
    <td>우리은행</td>
    <td>1층(운영시간) 09:30~15:30</td>
    <td>02-3158-3656</td>
  </tr>
  <tr>
    <td>블루포트(take out coffee)</td>
    <td>1층 중앙로비</td>
    <td>-</td>
  </tr>
</tbody>
</table>
---------
답 : 
---------
편의시설 : 식당
위치 : 1층 학생식당 내
전화번호 : -

편의시설 : 편의점(GS)
위치 : 1층 학생식당 내
전화번호 : 02-3158-0336

편의시설 : 우리은행
위치 : 1층(운영시간) 09:30~15:30
전화번호 : 02-3158-3656

편의시설 : 블루포트(take out coffee)
위치 : 1층 중앙로비
전화번호 : -
---------

예시 #3
테이블:
---------
<table class="table_nrm">
    <tbody><tr>
        <th colspan="2">구분</th>
        <th>비고</th>
    </tr>
    <tr>
        <td class="ta_center">공통서류</td>
        <td class="ta_center">전과신청서 각 1부<br>성적증명서 1부</td>
        <td class="ta_center" rowspan="3">전과 희망 학부(과) 사무실</td>
    </tr>
    <tr>
        <td class="ta_center">경영학부<br>항공교통물류학부</td>
        <td class="ta_center">토익성적증명서 1부</td>
    </tr>
    <tr>
        <td class="ta_center">항공운항학과</td>
        <td class="ta_center">전과지원서 1부<br>토익성적증명서 1부<br>항공신체검사증명서 1부</td>
    </tr>
</tbody></table>
---------
답 : 
---------
구분 : 공통서류 - 전과신청서 각 1부, 성적증명서 1부
비고 : 전과 희망 학부(과) 사무실

구분 : 경영학부, 항공교통물류학부 - 토익성적증명서 1부
비고 : 전과 희망 학부(과) 사무실

구분 : 항공운항학과 - 전과지원서 1부, 토익성적증명서 1부, 항공신체검사증명서 1부
비고 : 전과 희망 학부(과) 사무실
---------

시작!
테이블:
---------
{table_g}
---------
답 : 
"""
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-16k-0613",
  messages=[
  {"role": "system", "content": f'"{sys}"'},
])
answer = completion['choices'][0]['message']['content']
if answer.endswith('"'):
    answer = answer[:-1]
    answer = answer
print(answer)

In [ ]:
ass1 = """
함수 이름: [table_to_text]
입력: ["<table></table>"]
규칙: [당신이 정리하는 역할을 하길 바랍니다. 나는 <table> 태그를 포함한 입력 양식을 제공할 것이며 당신은 <table>을 감지하고 텍스트로 정리할것입니다.]
table_to_text(<table class="table_nrm center">
                                    <tbody><tr>
                                        <th rowspan="2">증명서 종류</th>
                                        <th rowspan="2">발급방법</th>
                                        <th colspan="2">수수료</th>
                                        <th rowspan="2">발급 시 관련부서</th>
                                    </tr>
                                    <tr>
                                        <th>국문</th>
                                        <th>영문</th>
                                    </tr>
                                    <tr>
                                        </tr><tr>
                                            <td>성적/재학/휴학/재적증명서<br>제적/졸업예정/졸업/수료증명서</td>
                                            <td>인터넷발급(<a style="color: blue;" href="http://hangkong.certpia.com/" target="_blank"><b>바로가기클릭☞</b></a>), <br>학생회관발급기<br>주민센터 내 FAX민원, <br>우체국 우편발급</td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>재학 1,200원<br>졸업 1,500원</td>
                                            <td>교무팀, 대학원행정실</td>
                                        </tr>
                                        <tr>
                                            <td>학적부, 사실증명원</td>
                                            <td>방문발급</td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>영문발급 無</td>
                                            <td>교무팀, 대학원행정실</td>
                                        </tr>
                                        <tr>
                                            <td>장학수혜확인서</td>
                                            <td>인터넷발급(<a style="color: blue;" href="http://hangkong.certpia.com/" target="_blank"><b>바로가기클릭☞</b></a>), <br>학생회관발급기<br>* 대학원은 방문 발급만 해당</td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>재학 1,200원<br>졸업 1,500원</td>
                                            <td>학생지원팀, 대학원행정실</td>
                                        </tr>
                                        <tr>
                                            <td>교육비납입증명서</td>
                                            <td>종합정보시스템에서 출력가능</td>
                                            <td>없음</td>
                                            <td>영문발급 無</td>
                                            <td>재무팀</td>
                                        </tr>
                                        <tr>
                                            <td>교외장학지원 관련 추천서</td>
                                            <td>방문발급</td>
                                            <td>없음</td>
                                            <td>영문발급 無</td>
                                            <td>학생지원팀, 대학원행정실</td>
                                        </tr>
                                        <tr>
                                            <td>실무경력확인서</td>
                                            <td>인터넷발급<br>학생회관발급기, 방문발급<br></td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>영문발급 無</td>
                                            <td>항공교통관제교육원</td>
                                        </tr>
                                        <tr>
                                            <td>비행경력증명서</td>
                                            <td>인터넷발급<br>학생회관발급기, 방문발급<br></td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>영문발급 無</td>
                                            <td>비행교육원</td>
                                        </tr>
                                        <tr>
                                            <td>비행교육원 이수증명서</td><td>인터넷발급<br>학생회관발급기, 방문발급<br></td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>영문발급 無</td>
                                            <td>비행교육원</td>
                                        </tr>
                                        <tr>
                                            <td>공학교육인증 참여확인서</td>
                                            <td>인터넷발급<br>학생회관발급기, 방문발급</td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>영문발급 無</td>
                                            <td>공학교육혁신센터</td>
                                        </tr>
                                        <tr>
                                            <td>항공정비사 정비실습확인서</td>
                                            <td>인터넷발급<br>학생회관발급기, 방문발급<br></td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>영문발급 無</td>
                                            <td>항공우주 및 기계공학부</td>
                                        </tr>
                                        <tr>
                                            <td>운항관리경력증명서</td>
                                            <td>인터넷발급<br>학생회관발급기, 방문발급<br></td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>영문발급 無</td>
                                            <td>비행교육원<br></td>
                                        </tr>
                                </tbody></table>)
"""

In [ ]:
import os
import openai

openai.api_key ="API_KEY"


# 테이블 태그 임의로 지정하기
table_g = """<table class="table_nrm center">
<tbody>
  <tr>
    <th>편의시설</th>
    <th>위치</th>
    <th>전화번호</th>
  </tr>
  <tr>
    <td>식당</td>
    <td>1층 학생식당 내</td>
    <td>-</td>
  </tr>
  <tr>
    <td>편의점(GS)</td>
    <td>1층 학생식당 내</td>
    <td>02-3158-0336</td>
  </tr>
  <tr>
    <td>우리은행</td>
    <td>1층(운영시간) 09:30~15:30</td>
    <td>02-3158-3656</td>
  </tr>
  <tr>
    <td>블루포트(take out coffee)</td>
    <td>1층 중앙로비</td>
    <td>-</td>
  </tr>
</tbody>
</table>"""


# (텍스트 + 테이블 태그)를 입력으로 주었을 때 문제가 발생하는지 파악
full = f"""               

사회봉사 과목이수안내
1. 사회봉사 이수방법
이론교육(4시간) + 봉사활동(34시간 이상)

{table_g}

※ 수강신청 시 본인 최대수강신청 학점에 미포함
※ 재학 중 1회 1학점씩 총 2회 2학점까지 이수 가능
※ 성적산출시 취득학점에 포함, 평점평균 산정에 미포함
※ 연속 2회 “NP”일 경우 수강신청 불가

3. 사회봉사활동 인정기간
수강신청 직전학기 기말시험 시작일 ~ 수강신청 학기 기말시험 전까지(※ 헌혈은 기간제한 없음)

4. 사회봉사활동 기관 인정 범위
가. 시·도 및 시·군·구 자원봉사센터, 청소년활동진흥센터, 기타 사회복지관, 지역아동센터, 사회복지법인 사회복지봉사활동인증관리(VMS) 및
       1365자원봉사포털에 등록되어 ‘사회복지 봉사활동 실적 인증서(VMS)’ 및 ‘자원봉사활동 확인서(1365)’가 출력되는 기관
VMS 출력 시 필히 ‘봉사자 실적 내역정보’ 출력하여 제출
니. 비영리민간단체(봉사활동확인서 제출 시 ‘비영리민간단체등록증’ 사본 첨부)
다. 해외봉사 : 인증된 국제 NGO단체
NGO(Non-Governmental Organization 비정부기구)는 지역-국가-국제적으로 조직된 자발적인 비영리 시민단체
라. 자원봉사센터 및 시민단체와 연계하여 하는 농촌봉사활동
마. 총학생회 주관 자원봉사
1학기 : 항공산타, 신입생새로배움터, 은익제 등
2학기 : 항공제, 하계방학 중 실시 한 사회봉사활동 등
바. 기타 부서 주관 대외행사 및 교내 자원봉사
입학관리팀 ‘모교입시 설명회’, 국제교류부 ‘Global-Buddy’, 항공우주박물관 ‘안내봉사’ 등
사. 헌혈활동 : 헌혈증 제출
1회 기준 4시간 인정
1학기 최대 2회(8시간)까지만 인정
추후 본교 재학생 중 필요한 학생 및 저소득층, 백혈병 소아암 협회, 관련 사회복지기관 등에게'항공대 재학생' 이름으로 기증

5. 사회봉사활동 미인정 범위
가. 공공기관, 행정기관에서의 봉사활동
공공기관 : 주민센터(구. 동사무소), 시.도.군.구청, 경찰서, 세무서, 소방서, 우체국 등
정부행정기관 : 국세청, 법원, 감사원, 국회 등
나. 도서관(단, 시각장애인을 위한 점자도서관은 인정), 어린이집, 경로당, 종교관련 봉사활동
다. 시·군·구 개최, 지역 개최 봉사활동(단, 교육과학기술부에서 공문화 되어 온 봉사활동은 인정)
라. 제출기간까지 봉사활동확인서가 미발급되거나 기간 및 시간이 정확히 기재되지 않은 봉사활동
   (예 : 중앙일보 '공부의 신')

6. 제출서류
가. “사회봉사”과목 학점인정 서류
수강신청 전원 제출, 미제출시 학점인정 안됨
본교 총학생회 주관 자봉단 활동 시에도 제출
나. 사회봉사활동확인서(봉사활동을 실시한 기관에 양식이 없을 경우 사용)

7. 유의사항
가. 필히 ‘이론교육’ 4시간을 이수해야 봉사활동확인서 제출 시 학점 인정
나. 봉사활동 확인서는 원본을 제출하여야 하며, 제출한 서류는 일체 반환하지 않음
다. 봉사활동 확인서 수정 금지 (수정 시 담당자 날인이 없을 경우 무효 처리)
    * 문의사항 : 02-300-0014
"""


# 테이블과 관련된 질문하기
question ="""혹시 편의시설에 대해 알려줄래?"""



sys = f"""{full}

{question}"""

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-16k-0613",
  messages=[
  {"role": "system", "content": f'"{sys}"'},
])
answer = completion['choices'][0]['message']['content']
# answer = answer.replace('\"',"")
print(answer)

In [ ]:
import pandas as pd
import openai
import csv
import numpy as np
from time import sleep
import json
import ast
import re
import random
from itertools import *
import sys
delay = 2

openai.api_key ="API_KEY"

def get_embedding(text, model="text-embedding-ada-002"):
    count = 0 # 예외 발생 횟수를 저장할 변수
    max_count = 5 # 예외가 발생한 최대 횟수
    while True:
        try :
            temp = openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']
            count=0
            return temp
        except Exception as e:
            count += 1
            print(count)
            if count >= max_count:
                print("예외가 연속으로 %d번 발생했습니다. 프로그램을 종료합니다." % max_count)
                break
            else:
                print("예외가 발생했습니다. (%s)" % e)
def split_sentences(text):
    sentences = text.split('\n')  # newline('\n')을 기준으로 문자열을 나눕니다.
    sentences = [sentence.strip() for sentence in sentences]  # 각 문장 앞뒤에 있는 공백을 제거합니다.
    sentences = [sentence for sentence in sentences if sentence]  # 빈 문장을 제거합니다.
    return sentences

                
Sec_index = []
SecList = []
    
csv_name = "test3_filter"

data = pd.read_csv(f'{csv_name}.csv', header=None, encoding="UTF-8")
for i in range(len(data)):    
    Sec_index.append(data.iloc[i][0])
    SecList.append(data.iloc[i][1])
    
###########################################################################################
temp_str = ""
for i in range(len(Sec_index)):
    print(f"------------  {i}  -------------")
    temp_manual = SecList[i]
    splitList = split_sentences(temp_manual)
    for j in range(len(splitList)):
        pattern = re.compile(r'\d+\.')
        matches = re.findall(pattern, temp_sentence)
        if matches:
            temp_str = temp_str + splitList[j]
        print(splitList[j])
        print("######################")
        